In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings('ignore')

In [2]:
p1 = pd.read_csv('Output/ensemble_f50.csv', index_col='customer_ID')
# p2 = pd.read_csv('archive_models/models_DART_all_56corr_pcaslope_lag_SEED42/p_M6_folds.csv', index_col='customer_ID')

In [3]:
p1['prediction'] *= .99 

In [4]:
p1.to_csv('Output/ensemble_f50_multipleby0.99.csv')

In [3]:
p2_std = p2.std(axis = 1).sort_values(ascending=False)

In [4]:
indx = p2_std[p2_std > 0.05].index

In [5]:
p2_votes = p2.loc[indx].groupby('customer_ID').apply(lambda x: pd.DataFrame(data = GaussianMixture(n_components=3).fit(x.values.reshape(-1, 1)).means_).T)
p2_votes = pd.DataFrame(p2_votes.values, index = indx, columns = ['vote_1', 'vote_2', 'vote_3'])
p2_votes

,vote_1,vote_2,vote_3
customer_ID,,,
e1e3b9dc555f07ee53ee3c2fca78e7dc153e06fa7ffda798845a77d0982336fc,0.418828,0.545950,0.461052
a7dde773746a793a4b9c2c9a22f403c47d7fabf3e7793cebabc602c5c0524bba,0.631406,0.705776,0.790278
88db3baa19d3d9eaa7858f8aa75fe8c218cc84f6a5ace992535128576fb82cdd,0.647895,0.554630,0.711526
0525c054db8f8b450a50ac7712255d35dcf9db68b3595fd2e87b3c507cfe1f43,0.308797,0.432849,0.342302
d1d6209ad36ee11703306596951067bc6a186102f891187133e76c10c8fd6d56,0.625043,0.537654,0.488537
...,...,...,...
51e857d3219870f13b076336418225becf07a0383906e9ff4bbcb5e6bcd87a2e,0.731065,0.683466,0.573236
7e2eec4634fe4f1ee4a83ffc24ca9c7147a15fc8c4feb0f700f68a92664dfed9,0.202995,0.325087,0.240033
df68a6793edd91d7b574070324c0f6431addcad27f9409434877cae671d99d38,0.253413,0.172584,0.299767


In [14]:
p2_random = p2_votes.groupby('customer_ID').apply(lambda x: x.sample(axis = 1)).sum(axis = 1)
p2_random.rename('prediction', inplace = True)

customer_ID
e1e3b9dc555f07ee53ee3c2fca78e7dc153e06fa7ffda798845a77d0982336fc    0.545950
a7dde773746a793a4b9c2c9a22f403c47d7fabf3e7793cebabc602c5c0524bba    0.790278
88db3baa19d3d9eaa7858f8aa75fe8c218cc84f6a5ace992535128576fb82cdd    0.711526
0525c054db8f8b450a50ac7712255d35dcf9db68b3595fd2e87b3c507cfe1f43    0.308797
d1d6209ad36ee11703306596951067bc6a186102f891187133e76c10c8fd6d56    0.625043
                                                                      ...   
51e857d3219870f13b076336418225becf07a0383906e9ff4bbcb5e6bcd87a2e    0.573236
7e2eec4634fe4f1ee4a83ffc24ca9c7147a15fc8c4feb0f700f68a92664dfed9    0.325087
df68a6793edd91d7b574070324c0f6431addcad27f9409434877cae671d99d38    0.253413
6886fa21c7196da0d378a970d31e3de4715b7fabc6d5a9602cc32abcc43b23b4    0.419452
843ac9151a30b9b62467caaa59b344b0a49e01c0c0b686a19e5f7074acf4f5d7    0.480676
Name: prediction, Length: 20685, dtype: float64

In [7]:
p1.drop(indx, inplace=True)

In [26]:
pd.concat([p1, p2_votes['vote_3']], axis=1).sum(axis=1).to_csv('Output/e4.csv', header=['prediction'])